In [38]:
import bt
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta

from data.get_data import get_prices_df, get_pdf_df, get_base_price_df
from data.fetch_data import fetch_index_info

from core.get_backtest import get_base_backtest, get_eql_backtest, get_mkw_backtest, get_bdd_mkw_backtest
from core.get_weigh import get_cap_weigh, get_bdd_cap_weigh


class CONFIG:
    ticker = "BOTZ"
    
    upper_bound=fetch_index_info(etf_tkr=ticker)[0]['upper_bound']
    
    start_date = datetime.now() - relativedelta(years=1)
    
    template="simple_white"
    primary_color="#F58220"
    secondary_color="#CDCECB"
    
    
def get_miraeasset_theme_figure(data, title):
    fig = px.line(data, 
              template=CONFIG.template,
              title=title,
              labels={
                  "value": "YTD",
                  "index": "date",
                  "variable": "ETFs"
              })

    fig['data'][0]['line']['color']=CONFIG.primary_color
    fig['data'][1]['line']['color']=CONFIG.secondary_color
    
    return fig



etf_price = get_base_price_df(etf_tkr=CONFIG.ticker, start_date=CONFIG.start_date).dropna()

pdf_df = get_pdf_df(etf_tkr=CONFIG.ticker)

tkrs = pdf_df.child_stk_tkr.to_list()

child_prices = get_prices_df(tickers=tkrs, start_date=CONFIG.start_date)

### 01 동일가중
- 리밸런싱: 3개월

In [39]:
base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)
eql_backtest = get_eql_backtest(name=f"{CONFIG.ticker} 동일가중", child_prices=child_prices)

ret = bt.run(eql_backtest, base_backtest)

TITLE=f"{CONFIG.ticker} vs {CONFIG.ticker} 동일가중"

fig = get_miraeasset_theme_figure(data=ret._get_series(freq=None).rebase(),
                                  title=TITLE)

fig.show()

### 02 시가총액가중(유동시가총액)
- 리밸런싱: 3개월

In [41]:
quartely_child_cap_weigh = get_cap_weigh(child_prices=child_prices, pdf_df=pdf_df)

base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)
mkw_backtest = get_mkw_backtest(name=f"{CONFIG.ticker} 시총가중",
                                child_prices=child_prices,
                                weigh=quartely_child_cap_weigh)

ret = bt.run(mkw_backtest, base_backtest)

TITLE=f"{CONFIG.ticker} vs {CONFIG.ticker} 시총가중"

fig = get_miraeasset_theme_figure(data=ret._get_series(freq=None).loc[quartely_child_cap_weigh.index[0]:].rebase(),
                                  title=TITLE)

fig.show()

### 03 최대제한 시가총액가중(유동비율)
- 리밸런싱 3개월

In [42]:
quartely_child_cap_upper_bound_weigh = get_bdd_cap_weigh(child_prices=child_prices,
                                                         pdf_df=pdf_df,
                                                         upper_bound=CONFIG.upper_bound)

base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)
bdd_mkw_backtest = get_bdd_mkw_backtest(name=f"{CONFIG.ticker} 최대제한 시총가중",
                                child_prices=child_prices,
                                weigh=quartely_child_cap_upper_bound_weigh)

ret = bt.run(bdd_mkw_backtest, base_backtest)

TITLE=f"{CONFIG.ticker} vs {CONFIG.ticker} 최대제한 시총가중"

fig = get_miraeasset_theme_figure(data=ret._get_series(freq=None).loc[quartely_child_cap_upper_bound_weigh.index[0]:].rebase(),
                                  title=TITLE)

fig.show()

In [43]:
x_axis = ret._get_series(freq=None).index.to_list()
x_axis[:5]

[Timestamp('2022-09-05 00:00:00'),
 Timestamp('2022-09-06 00:00:00'),
 Timestamp('2022-09-07 00:00:00'),
 Timestamp('2022-09-08 00:00:00'),
 Timestamp('2022-09-09 00:00:00')]

### 04 (2번 통신 response) API 통신을 위한 JSON 형식 변환

In [44]:
import pandas as pd


weighs = pd.DataFrame()

weighs['mkw_ratio'] = quartely_child_cap_weigh.iloc[-1].round(4)
weighs['umkw_ratio'] = quartely_child_cap_upper_bound_weigh.iloc[-1].round(4)
weighs['eql_ratio'] = 1/weighs.shape[0]
weighs['eql_ratio'] = weighs['eql_ratio'].round(4)

weighs = weighs.reset_index().rename(columns={"index": "child_stk_tkr"})

weighs['child_stk_tkr'] = weighs['child_stk_tkr'].str.upper()

pdf = get_pdf_df(etf_tkr=CONFIG.ticker)

weighs = weighs.merge(pdf[['child_stk_tkr', 'child_stk_name']], on='child_stk_tkr', how='left')

# weighs.T.to_dict()

### 05 (3번 통신 response) API 통신을 위한 누적수익률 및 리밸런싱 내역 추출
- 누적수익률 추출(myEtfYtd)

In [45]:
base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)

user_weigh = quartely_child_cap_upper_bound_weigh.copy()
user_etf_name = f"{CONFIG.ticker} 김밍두기요오"
user_backtest = get_bdd_mkw_backtest(name=user_etf_name,
                                    child_prices=child_prices,
                                    weigh=user_weigh)

ret = bt.run(user_backtest, base_backtest)

cumulative_ret = ret._get_series(freq=None).loc[quartely_child_cap_upper_bound_weigh.index[0]:].rebase()
 
cumulative_ret = cumulative_ret[user_etf_name].reset_index().rename(columns={"index": "date", user_etf_name: "ytd"})
cumulative_ret["date"] = cumulative_ret["date"].apply(lambda x: str(x).split(' ')[0])

cumulative_ret["ytd"].to_list()[:10]

[100.0,
 102.75132757582664,
 106.4457758152008,
 105.4800745423317,
 104.92233105640409,
 100.29855182785987,
 98.45583864994047,
 98.44932364788053,
 98.12953906159399,
 99.81169893126486]

- 리밸런싱 내역 추출

In [46]:
rebalance = ret.get_weights().loc[quartely_child_cap_upper_bound_weigh.index[0]:].drop(user_etf_name, axis=1)
rebalance.columns = rebalance.columns.str.replace(user_etf_name+">", "")

for col in rebalance.columns:
    print(rebalance[col].to_list()[:10])
    break

[0.07984944104003906, 0.08016573819858706, 0.08065706363862309, 0.08133088875060415, 0.08026908843139989, 0.08013842215571884, 0.08071669734347123, 0.08044077345127323, 0.08089392776023906, 0.08125488589862474]


In [47]:
import plotly.express as px

wei_df = round(ret.get_security_weights().loc[quartely_child_cap_upper_bound_weigh.index[0]:], 3)

fig = px.area(wei_df,
              template='simple_white',
              color_discrete_sequence=px.colors.sequential.Oranges)

for d in fig.data:
    d.fillcolor = d.line.color

fig.show()

- 낙폭(Drawdown) 추출

In [48]:
drawdown_series = ret._get_series(freq=None).loc[quartely_child_cap_upper_bound_weigh.index[0]:].to_drawdown_series()
drawdown_series[user_etf_name].to_list()[:10]

[0.0,
 0.0,
 0.0,
 -0.009072236690215396,
 -0.014311932503940139,
 -0.05774981618822583,
 -0.07506110133606014,
 -0.07512230622662575,
 -0.07812650797946674,
 -0.062323533584397794]